# Remora ionic current analysis for tRNAs

This notebooks aims to apply signal alignment using Remora's 'dwell_penalty' algorithm with subsequent statistical comparison of trimmed mean signal levels per base positions, anchored on the reference sequences.
Content of this notebook:
1. Import of required python packages
2. Functions
3. Data loading
4. Signal Alignment
5. Statistical comparison
6. Visualisation

This is mainly based on the Remora API notebooks (https://github.com/nanoporetech/remora/tree/master/notebooks)

## 1. Import of required python packages

In [ ]:
# ---- General packages ----
import logging
import pickle
import random
import numpy as np
import pandas as pd
import pingouin as pg
from datetime import datetime
from pathlib import Path
import polars as pl
import patchworklib as pw

# ---- For plotting ----
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sns


# ---- Statistical testing ----
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

# ---- Specific packages for input ----
import pod5
from pod5 import DatasetReader
import pysam
from remora import io, refine_signal_map, util

# Silence Remora DEBUG messages
logging.getLogger("Remora").setLevel(logging.INFO)


## 2. Functions

### Analysis functions

#### For bootstrapping the IVT sample

Due to unequal sample sizes, we used bootstrapping in which multiple samples of the IVT-tRNA sample were drawn with comparable sample size to the biological sample.

In [ ]:
class FilteredReadIndexedBam(io.ReadIndexedBam):
    def __init__(self, bam_path, read_names_to_keep):
        super().__init__(bam_path)
        self.read_names_to_keep = read_names_to_keep

    def fetch(self, *args, **kwargs):
        for read in super().fetch(*args, **kwargs):
            if read.query_name in self.read_names_to_keep:
                yield read

In [ ]:
def take_samples_read_ids(
    bam_path,
    sample_size=60,
    number_of_samples=50
):
    """
    Aim: create subsamples based on sample drawing of read-IDs.

    Parameters:
        - bam_path: the file path of the bam file of the IVT sample
        - sample_size: the required subsample size, comparable to biological sample
        - number_of_samples: the required number of subsamples
    """
    
    # Open BAM and get all read names
    bam_file = pysam.AlignmentFile(bam_path, "rb")
    all_read_names = [read.query_name for read in bam_file if not read.is_secondary]
    bam_file.close()

    # Ensure the pool is large enough
    if sample_size > len(all_read_names):
        raise ValueError("Sample size exceeds total number of unique reads available.")

    # Create multiple subsamples
    samples = [
        random.sample(all_read_names, k=sample_size)  # without replacement
        for _ in range(number_of_samples)
    ]

    print(f"Generated {len(samples)} samples, each with {sample_size} unique read IDs.")
    return samples

#### MWU testing

In [ ]:
def make_MWU_df(
    ref_reg,
    samples_metrics_1,
    samples_metrics_2,
    min_reads = 30,
    ):
    """
    Make a dataframe in which Mann-Whitney U test is applied per position with FDR-correction.

    Parameters:
        ref_reg: the tRNA reference region of interest
        samples_metrics_1/2: metric dicts for each sample, including the trimmmed means
        min_reads: the minimum amount of reads used in analysis (only testing for significance for positions with enough reads)
    """
  
    trimmean_data_1= pd.DataFrame(samples_metrics_1[0]['trimmean'])
    trimmean_data_1.columns = range(ref_reg.start, ref_reg.end)
    print(f"The number of reads in sample 1: {trimmean_data_1.shape[0]}, the number of positions: {trimmean_data_1.shape[1]}")
    trimmean_data_2= pd.DataFrame(samples_metrics_2[0]['trimmean'])
    trimmean_data_2.columns = range(ref_reg.start, ref_reg.end)
    print(f"The number of reads in sample 2: {trimmean_data_2.shape[0]}, the number of positions: {trimmean_data_2.shape[1]}")

    results = []
    for col in trimmean_data_1.columns:
        ### Per position drop the reads for which no signal value exists
        data_1=  trimmean_data_1[col].dropna()
        data_2 =  trimmean_data_2[col].dropna()
        ### When both samples have at least the minimum amount of reads calculate the Mann-Whitney-U test
        if (len(data_1) >= min_reads) and (len(data_2) >= min_reads):
            result_mwu = pg.mwu(data_1, data_2, alternative='two-sided')
            stat = result_mwu['U-val'].values[0]
            pval = result_mwu['p-val'].values[0]
            RBC = result_mwu['RBC'].values[0]

            results.append({'position': col, 
                            'num_sample_1': len(data_1),
                            'num_sample_2': len(data_2),
                            'median_trimmean_1': np.median(data_1),
                            'median_trimmean_2': np.median(data_2),
                            'sd_trimmean_1': np.std(data_1, ddof= 1),
                            'sd_trimmean_2': np.std(data_2, ddof= 1),
                            'p_value': pval,
                            'RBC': RBC})

    results_df = pd.DataFrame(results)

    ### Adjust the p-values for multiple hypotheses testing
    pvals = results_df['p_value']
    _, pvals_corrected, _, _ = multipletests(pvals, method='fdr_bh', alpha = 0.01)

    results_df['adjusted_p_value'] = pvals_corrected
    return results_df

In [ ]:
def compute_MWU_results(
    ref_reg,
    samples_metrics_1,
    samples_metrics_2,
    threshold = 0.01,
    min_reads = 30
):
    """
    Filter the positions for which Mann_Whitney_U testing results significant, using make_MWU_df()

    Parameters:
        ref_reg: the tRNA reference region of interest
        samples_metrics_1/2: metric dicts for each sample, including the trimmmed means
        treshold: the alpha-level for the p-value
        min_reads: the minimum amount of reads used in analysis (only testing for significance for positions with enough reads)
        max_reads: the maximum maximum amount of reads used in analysis
    """

    df_Mann_Whitney_U = make_MWU_df(ref_reg, samples_metrics_1, samples_metrics_2, min_reads)
    ### Convert the generated output into a pandas dataframe
    df_Mann_Whitney_U_pd = pd.DataFrame(df_Mann_Whitney_U)

    ### Filter for only significant positions
    filtered_df = df_Mann_Whitney_U_pd[(df_Mann_Whitney_U_pd['adjusted_p_value'] < threshold)]
    print(filtered_df.shape)

    significant_positions = list(filtered_df['position'])

    print(f"Number of significantly different positions for this entry: {len(significant_positions)}")
    
    if len(significant_positions) > 0:
        return filtered_df, significant_positions
    else:
        return None, None

### Plotting functions

#### Line plot

In [ ]:
def plot_trimmean_signals_giving_metrics_one_sample_with_seq(
    ref_reg,
    samples_metrics,
    xlim_start=0,
    xlim_end=20,
    signal_type='norm',
    max_reads=None,
    alpha_lines=1/200,
    color = "#009E73",
    highlight_positions=None,
    sequence=None 
):
    """
    Plot trimmean signals for 1 sample per position with sequence shown on x-axis.

    Parameters:
        ref_reg: object with .start, .end, .ctg
        samples_metrics: metric dicts for the sample
        xlim_start/xlim_end: x-axis boundaries
        signal_type: 'norm' or 'pa'
        max_reads: maximum numbers of reads to plot
        alpha_lines: transparency
        color: color of the lines
        highlight_positions: list of positions to highlight
        sequence: (optional) string of nucleotides matching region length
    """
    ## Getting the trimmed mean signal levels of the mapped reads
    trimmean_data = pd.DataFrame(samples_metrics[0]['trimmean'])
    trimmean_data.columns = range(ref_reg.start, ref_reg.end)
    print(f"Shape of metric array: {samples_metrics[0]['trimmean'].shape}")

    # Limit x-axis range
    plot_positions = [pos for pos in trimmean_data.columns if xlim_start <= pos - ref_reg.start < xlim_end]


    plt.figure(figsize=(10, 6))
    
    # If too many reads, draw a random sample
    if max_reads is not None and len(trimmean_data) > max_reads:
        data_to_plot = trimmean_data.sample(n=max_reads, random_state=42)
    else:
        data_to_plot = trimmean_data

    # plot each read as a line
    for i in range(len(data_to_plot)):
        plt.plot(plot_positions, data_to_plot[plot_positions].iloc[i], color=color, alpha=alpha_lines)

    print(f"Plotting {len(data_to_plot)} reads")

    # Show both sequence and position if sequence is provided
    if sequence is not None:
        print(f"The sequence being plotted is {len(sequence[xlim_start:xlim_end])} nt")
        
        ax = plt.gca()  # Get current axis

        # x-axis tick positions
        x_positions = list(range(xlim_start, xlim_end))

        if sequence is not None:
            sequence_slice = sequence[xlim_start:xlim_end]
            if len(sequence_slice) != (xlim_end - xlim_start):
                raise ValueError("Length of sequence slice does not match expected range.")

            # Combine sequence and index labels with newline
            combined_labels = [f"{base}\n{idx}" for idx, base in zip(x_positions, sequence_slice)]
            ax.set_xticks(x_positions)
            ax.set_xticklabels(combined_labels)
            ax.set_xlabel("Base (top) and index (bottom)")
        else:
            ax.set_xticks(x_positions)
            ax.set_xticklabels(x_positions)
            ax.set_xlabel("Sequence index")

    else:
        plt.xticks(plot_positions)

    # Adjust the y-axis to the signal type
    if signal_type == 'norm':
        plt.ylim(-4, 4)
        plt.ylabel('Trimmean (norm)')
    elif signal_type == 'pa':
        plt.ylim(25, 140)
        plt.ylabel('Trimmean (pA)')

    plt.title(ref_reg.ctg)
    plt.grid(color='gray', linestyle='-', linewidth=0.3, alpha=1/30)

    if highlight_positions is not None:
        for position in highlight_positions:
            plt.axvspan(position - 0.5, position + 0.5, color='yellow', alpha=0.3)

    plt.tight_layout()
    plt.show()


In [ ]:
def plot_trimmean_signals_two_samples_with_seq(
    ref_reg,
    samples_metrics_1,
    samples_metrics_2,
    xlim_start=0,
    xlim_end=20,
    signal_type='norm',
    max_reads=None,
    alpha_lines=1/200,
    sample_labels=None,
    highlight_positions= None,
    sequence= None
):
    """
    Plot trimmean signals for 2 samples on the same plot.
    
    Parameters:
        ref_reg: object with .start, .end, .ctg
        samples_metrics_1/2/3: metric dicts for each sample
        xlim_start/xlim_end: x-axis boundaries, define region
        signal_type: 'norm' or 'pa'
        max_reads: maximum numbers of reads to plot
        alpha_lines: transparency
        sample_labels: list of 3 labels
        highlight_positions: list of positions to highlight
        sequence: (optional) string of nucleotides matching region length
    """
    trimmean_data_1= pd.DataFrame(samples_metrics_1[0]['trimmean'])
    trimmean_data_1.columns = range(ref_reg.start, ref_reg.end)

    trimmean_data_2= pd.DataFrame(samples_metrics_2[0]['trimmean'])
    trimmean_data_2.columns = range(ref_reg.start, ref_reg.end)
    
    all_metrics = [trimmean_data_1,trimmean_data_2]
    colors = ['#009E73', "#7851A9"]
    if sample_labels is None:
        sample_labels = ['Sample 1', 'Sample 2']

    plt.figure(figsize=(10, 6))

    for idx, sample in enumerate(all_metrics):
        trimmean_data = all_metrics[idx]
        print(f"Shape of metric array {sample_labels[idx]}: {trimmean_data.shape}")

        # Downsample if needed
        if max_reads is not None and len(trimmean_data) > max_reads:
            data_to_plot = trimmean_data.sample(n=max_reads, random_state=42)
        else:
            data_to_plot = trimmean_data

        for i in range(len(data_to_plot)):
            plt.plot(
                data_to_plot.columns,
                data_to_plot.iloc[i],
                color=colors[idx],
                alpha=alpha_lines,
                label=sample_labels[idx] if i == 0 else ""
            )

        print(f"Plotting {len(data_to_plot)} reads for {sample_labels[idx]}")

    # Show both sequence and position if sequence is provided
    if sequence is not None:
        print(f"The sequence being plotted is {len(sequence[xlim_start:xlim_end])} nt")
        
        ax = plt.gca()  # Get current axis

        # x-axis tick positions
        x_positions = list(range(xlim_start, xlim_end))

        if sequence is not None:
            sequence_slice = sequence[xlim_start:xlim_end]
            if len(sequence_slice) != (xlim_end - xlim_start):
                raise ValueError("Length of sequence slice does not match expected range.")

            # Combine sequence and index labels with newline
            combined_labels = [f"{base}\n{idx}" for idx, base in zip(x_positions, sequence_slice)]
            ax.set_xticks(x_positions)
            ax.set_xticklabels(combined_labels)
            ax.set_xlabel("Base (top) and index (bottom)")
        else:
            ax.set_xticks(x_positions)
            ax.set_xticklabels(x_positions)
            ax.set_xlabel("Sequence index")

    else:
        plt.xticks(plot_positions)
        
    plt.xlim(xlim_start, xlim_end)
    # Adjust the y-axis to the signal type
    if signal_type == 'norm':
        plt.ylim(-4, 4)
        plt.ylabel('Trimmean (norm)')
    elif signal_type == 'pa':
        plt.ylim(25, 140)
        plt.ylabel('Trimmean (pA)')

    plt.title(ref_reg.ctg)
    plt.grid(color='gray', linestyle='-', linewidth=0.3, alpha=1/30)
    plt.legend()
    plt.tight_layout()

    if highlight_positions is not None:
        for position in highlight_positions:
            plt.axvspan(position-0.5, position+0.5, color='yellow', alpha=0.3)

    plt.show()

#### Scatterplot

In [ ]:
def plot_mean_and_sd_of_trimmean_two_samples(
    ref_reg,
    samples_metrics_1,
    samples_metrics_2,
    xlim_start,
    xlim_end,
    sample_labels,
    signal_type="norm",
    colors=None
):
    """
    Plot dotplots of the mean and sd of the trimmean values per position for 2 samples using matplotlib.
    
    Parameters:
        ref_reg: object with .start, .end, .ctg
        samples_metrics_1/2: metric dicts for each sample
        xlim_start/xlim_end: x-axis boundaries, define region
        sample_labels: list of 2 labels
        signal_type: 'norm' or 'pa'
        colors: colors to use for plotting
    """
    all_metrics = [samples_metrics_1, samples_metrics_2]
    all_means = []
    all_sds = []

    for i, sample in enumerate(all_metrics):
        # For each sample, get the the trimmed mean signal levels
        metrics = sample[0]  # Unpack dict
        trimmean_data = pd.DataFrame(metrics['trimmean'])

        # Calculate the mean per position
        trimmean_mean = trimmean_data.mean()
        df_mean = pd.DataFrame({
            'position': range(len(trimmean_mean)),
            'value': trimmean_mean,
            'label': sample_labels[i]
        })
        all_means.append(df_mean)

        # Calculate the std per position
        trimmean_sd = trimmean_data.std()
        df_sd = pd.DataFrame({
            'position': range(len(trimmean_sd)),
            'value': trimmean_sd,
            'label': sample_labels[i]
        })
        all_sds.append(df_sd)

    # Combine the samples
    df_all_means = pd.concat(all_means, ignore_index=True)
    df_all_sds = pd.concat(all_sds, ignore_index=True)

    # Set y-limits
    if signal_type == 'norm':
        mean_ylim = (-3, 3)
        sd_ylim = (0, 3)
        mean_yticks = np.arange(-3, 3.1, 0.5)
        sd_yticks = np.arange(0, 3.1, 0.2)
    elif signal_type == 'pa':
        mean_ylim = (30, 140)
        sd_ylim = (0, 20)
        mean_yticks = np.arange(30, 141, 10)
        sd_yticks = np.arange(0, 21, 2)
    else:
        raise ValueError("Invalid signal_type. Choose 'norm' or 'pa'.")

    if colors is None:
        colors = ['#009E73', "#7851A9"]

    x_ticks = list(range(xlim_start, xlim_end + 1, 5))

    # Filter by xlim
    df_plot_means = df_all_means.query("position >= @xlim_start and position <= @xlim_end")
    df_plot_sds = df_all_sds.query("position >= @xlim_start and position <= @xlim_end")

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # Plot Mean
    for i, label in enumerate(sample_labels):
        subset = df_plot_means[df_plot_means['label'] == label]
        axs[0].scatter(subset['position'], subset['value'], label=label, color=colors[i], s=15)

    axs[0].set_title(f"{ref_reg.ctg} - Mean Trimmean")
    axs[0].set_ylabel("Mean of trimmed mean")
    axs[0].set_ylim(mean_ylim)
    axs[0].set_yticks(mean_yticks)
    axs[0].legend()
    axs[0].grid(True, alpha = 0.5)

    # Plot SD
    for i, label in enumerate(sample_labels):
        subset = df_plot_sds[df_plot_sds['label'] == label]
        axs[1].scatter(subset['position'], subset['value'], label=label, color=colors[i], s=15)

    
    #axs[1].axhline(y=1, color='red', linestyle='--', linewidth=1, label='SD=1 threshold')

    axs[1].set_title(f"{ref_reg.ctg} - SD Trimmean")
    axs[1].set_xlabel("Reference Position")
    axs[1].set_ylabel("SD of trimmed mean")
    axs[1].set_ylim(sd_ylim)
    axs[1].set_yticks(sd_yticks)
    axs[1].set_xticks(x_ticks)
    axs[1].legend()
    axs[1].grid(True, alpha = 0.5)

    plt.tight_layout()
    plt.show()

    # Print SD summary
    print("Mean SD values per sample:")
    for i, df in enumerate(all_sds):
        print(f"{sample_labels[i]}: {np.mean(df['value']):.4f}")


#### Violin plot

In [ ]:
def plot_trimmean_violin_two_samples_with_seq(
    ref_reg,
    samples_metrics_1,
    samples_metrics_2,
    xlim_start=0,
    xlim_end=20,
    signal_type='norm',
    sample_labels=None,
    highlight_positions_one_ionic=None,
    highlight_positions_two_ionic = None,
    highlight_positions_three = None,
    highlight_positions_basecalled = None,
    highlight_positions_basecalled_one = None,
    highlight_positions_excluded = None,
    sequence=None,
    legend = 'yes', title = None
):
    """
    Plot split violin plots of event-level means per position for two samples.

    Parameters:
        samples_metrics_1, samples_metrics_2: lists of dicts with 'trimmean' DataFrames.
        xlim_start/xlim_end: x-axis boundaries, define region
        signal_type: str, 'norm' or 'pa'
        sample_labels: list[str], list of 2 labels
        highlight_positions_one_ionic: list[int], positions predicted by only this ionic current analysis
        highlight_positions_two_ionic: list[int], positions predicted by both ionic current analyses
        highlight_positions_three: list[int], positions predicted by both ionic current analyses and basecalled modification analysis
        highlight_positions_basecalled: list[int], positions predicted by only basecalled modification analysis
        highlight_positions_basecalled_one: list[int], positions predicted by this ionic current analysis and basecalled modification analysis
        highlight_positions_excluded: list[int], positions excluded from the analysis
        sequence: str or None,  optional sequence string for annotated x-axis.
        legend: str 'yes' or 'no', binary to indicate whether to plot the legend
        title: str or None, title of the figure
    """
    df1 = pd.DataFrame(samples_metrics_1[0]['trimmean'])
    df2 = pd.DataFrame(samples_metrics_2[0]['trimmean'])

    dfs = [df1, df2]
    dfs_long = []
    for df, label in zip(dfs, sample_labels):

        # Keep only positions within specified range
        cols_in_range = [pos for pos in df.columns if isinstance(pos, (int, float)) and xlim_start <= pos <= xlim_end]
        df = df.loc[:, cols_in_range]

        # Transform to long format
        df_long = df.melt(var_name="Position", value_name="Trimmean")
        df_long["Sample"] = label
        dfs_long.append(df_long)

    # Combine the samples
    df_all = pd.concat(dfs_long, ignore_index=True)
    
    # Ensure Position is a categorical variable with proper ordering
    full_positions = list(range(xlim_start, xlim_end + 1))
    df_all["Position"] = pd.Categorical(df_all["Position"], categories=full_positions, ordered=True)

    # Configure plot
    custom_palette = {
        sample_labels[0]: '#009E73',  # Biological
        sample_labels[1]: "#7851A9"   # IVT
    }
    fig_width = 10 if legend == 'yes' else 6
    plt.figure(figsize=(fig_width, 4))

    # Highlight positions
    def add_highlight(highlight_list, color, alpha):
        if highlight_list is not None:
            for pos in highlight_list:
                if pos in full_positions:
                    idx = full_positions.index(pos)
                    plt.axvspan(idx - 0.5, idx + 0.5, color=color, alpha=alpha)

    add_highlight(highlight_positions_one_ionic, 'yellow', 0.1)
    add_highlight(highlight_positions_two_ionic, 'orange', 0.2)
    add_highlight(highlight_positions_three, 'red', 0.2)
    add_highlight(highlight_positions_basecalled, 'lightskyblue', 0.3)
    add_highlight(highlight_positions_basecalled_one, 'limegreen', 0.2)
    add_highlight(highlight_positions_excluded, 'grey', 0.2)

    # Plot violins
    ax = sns.violinplot(
        data=df_all,
        x="Position",
        y="Trimmean",
        hue="Sample",
        palette=custom_palette,
        split=True,
        inner="quartile",
        linewidth=0.8,
        scale="width",
        bw=0.2
    )
    plt.xlim(-0.5, len(full_positions) - 0.5)

    # X-axis
    if sequence is not None:
        sequence_slice = sequence[xlim_start:xlim_end+1]
        combined_labels = [f"{base}\n{pos}" for base, pos in zip(sequence_slice, full_positions)]
        ax.set_xticks(range(len(full_positions)))
        ax.set_xticklabels(combined_labels, fontsize=14)
        ax.set_xlabel("Base (top) and position (bottom)", fontsize=14)
    else:
        ax.set_xlabel("Position", fontsize=14)

    # Y-axis
    plt.ylim(-4, 4)
    plt.ylabel(f"Mean signal level ({signal_type})", fontsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ytick_vals = np.arange(-4, 4.2, 0.5)
    ytick_labels = [str(val) if val % 1 == 0 else "" for val in ytick_vals]
    plt.yticks(ytick_vals, ytick_labels)

    # Title and grid
    plt.title(title if title is not None else ref_reg.ctg)
    plt.grid(color='gray', linestyle='-', linewidth=0.3, alpha=1/30)
    plt.grid(axis='y', linestyle='-', linewidth=0.3, alpha=0.7)

    # Plot legend
    sample_handles = [
        mpatches.Patch(color='#009E73', label='Biological'),
        mpatches.Patch(color='#7851A9', label='IVT')
    ]
    legend_handles = []
    if highlight_positions_one_ionic is not None:
        legend_handles.append(mpatches.Patch(color='yellow', alpha=0.1, label='Predicted by one ionic current analysis'))
    if highlight_positions_two_ionic is not None:
        legend_handles.append(mpatches.Patch(color='orange', alpha=0.2, label='Predicted by both ionic current analyses'))
    if highlight_positions_basecalled is not None:
        legend_handles.append(mpatches.Patch(color='lightskyblue', alpha=0.3, label='Predicted by modification basecalling'))
    if highlight_positions_three is not None:
        legend_handles.append(mpatches.Patch(color='red', alpha=0.2, label='Predicted by all three analyses'))
    if highlight_positions_basecalled_one is not None:
        legend_handles.append(mpatches.Patch(color='limegreen', alpha=0.2, label='Predicted by ionic + basecalling'))
    if highlight_positions_excluded is not None:
        legend_handles.append(mpatches.Patch(color='grey', alpha=0.2, label='Excluded from analyses'))

    if legend == 'yes':
        all_handles = sample_handles + legend_handles
        ax.legend(
            handles=all_handles,
            labels=[h.get_label() for h in all_handles],
            fontsize=12,
            bbox_to_anchor=(1.02, 1),
            ncol=2,
            loc='upper left',
            borderaxespad=0
        )
    elif legend == 'no':
        ax.legend_.remove()

    # Show
    plt.tight_layout()
    plt.show()


## 3. Data loading

For both samples, the following is required:
- Pod5 file including the raw (unbasecalled) reads generated by RNA004-ONT sequecning
- Bam file including the basecalled, map reads. When using BWA-MEM sequence alignment, the bam file needs to be prepare using the bam_preparation python script in order to include both sequence alignments tags and move table tags.

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")


data_root = Path("/path/to/data/")

print("IVT sample: ")
pod5_dr_IVT = pod5.DatasetReader(data_root / "IVT_reads.pod5") 
bam_fh_IVT = io.ReadIndexedBam(data_root/"IVT_selected_reads_primary_tagged.bam")

print("Biological sample")
pod5_dr_bio = pod5.DatasetReader(data_root/"bio_reads.pod5")
bam_fh_bio = io.ReadIndexedBam(data_root/"bio_selected_reads_primary_tagged.bam")


end_time = datetime.now()
print(f"Finished at: {end_time}")
print(f"Total time taken: {end_time - start_time}")

### Bootstrapping the IVT-sample

In [ ]:
bam_path = data_root/"IVT_selected_reads_primary_tagged.bam"
subsamples = take_samples_read_ids(
    bam_path,
    sample_size=360,
    number_of_samples=50
)
# Save the created subsamples as pickle for future usage and for usage in f5c's analysis
with open("subsamples_IVT.pkl", "wb") as f:
    pickle.dump(subsamples, f)


### Loading the generated subsamples

In [ ]:
with open("path/to/data/subsamples_IVT.pkl", "rb") as f:
    subsamples = pickle.load(f)

### Checking the input data

Define the tRNA-region of interest

In [ ]:
ref_reg = io.RefRegion(ctg="[contig_name]", strand="+", start=0, end=72)

Check whether all the reads stored in the bam file have corresponding entry within the pod5 file. This is required for downstream analysis.

In [ ]:
bam_reads_bio = io.get_reg_bam_reads(ref_reg, bam_fh_bio)
print(f"The number of bam reads for biological sample: {len(bam_reads_bio)}")
io_reads_bio = io.get_io_reads(bam_reads_bio, pod5_dr_bio)
print(f"The number of io reads for biological sample: {len(io_reads_bio)}")


bam_reads_IVT = io.get_reg_bam_reads(ref_reg, bam_fh_IVT)
print(f"The number of bam reads for IVT: {len(bam_reads_IVT)}")
io_reads_IVT = io.get_io_reads(bam_reads_IVT, pod5_dr_IVT)
print(f"The number of io reads for IVT: {len(io_reads_IVT)}")

## 4. Signal Alignment

### Define the signal map refiner
First the kmer_model table need to be loaded, which is obtained from the kmer_models repository of ONT (https://github.com/nanoporetech/kmer_models/tree/master)

In [ ]:
level_table = data_root/"9mer_levels_v1.txt"

Thereafter, the signal map refiner is defined. For this adjustments within the function rescale() in refine_signal_map.py (start at line 486) are made to enable more inclusive precise rescaling (scale_iters =1):
- dwell_filter_pctls is set to (2, 98)
- edge_filter_bases is set to 5
- min_levels is set to 0

In [ ]:
sig_map_refiner = refine_signal_map.SigMapRefiner(
    kmer_model_filename=level_table,
    do_rough_rescale=True,
    scale_iters=1,
    do_fix_guage=True,
    sd_params=(50, 15, 1.5),
    half_bandwidth = 10
)
print(sig_map_refiner)
print(f"The parameters of the dwell_penalty algorithm are set to: {sig_map_refiner.sd_params}")

# Assigning the 6th base within each k-mer as dominant in order to convert signals-per-kmer to signals-per-base
sig_map_refiner.center_idx = 5
print(f"The index of the central base within k-mer is set to: {sig_map_refiner.center_idx}")

### Calculating the trimmed mean signal levels

Subsequently, the set signal alignment refiner is applied and the dwell, trimmed mean and trimmed standard deviation of the signals per reference base position is calculated for all the mapped reads per sample.

#### For biological sample

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")
samples_metrics_bio, all_bam_reads = io.get_ref_reg_samples_metrics(
    ref_reg,
    [(pod5_dr_bio, bam_fh_bio)],
    metric="dwell_trimmean_trimsd",
    sig_map_refiner=sig_map_refiner, signal_type = 'norm'
)
print(f"Metrics computed: {', '.join(samples_metrics_bio[0].keys())}")
print(f"Shape of metric array: {samples_metrics_bio[0]['trimmean'].shape}")
# The rows are the reads, the columns are the reference positions
end_time = datetime.now()
print(f"Ended at: {end_time}")

#### For IVT sample

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")
samples_metrics_IVT, all_bam_reads = io.get_ref_reg_samples_metrics(
    ref_reg,
    [(pod5_dr_IVT, bam_fh_IVT)],
    metric="dwell_trimmean_trimsd",
    sig_map_refiner=sig_map_refiner, signal_type = 'norm'
)
print(f"Metrics computed: {', '.join(samples_metrics_IVT[0].keys())}")
print(f"Shape of metric array: {samples_metrics_IVT[0]['trimmean'].shape}")
# The rows are the reads, the columns are the reference positions
end_time = datetime.now()
print(f"Ended at: {end_time}")

## 5. Statistical comparison

The trimmed mean signal levels between the biological and IVT sample are compared per reference position using Mann-Whitney-U testing.

### MWU testing with bootstrapping

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")

all_MWU_results = []
# For each subsample of IVT-sample a MWU-test is conducted with the biological sample.
for i, subsample in enumerate(subsamples):
    # subsample is a list of 50 read ids
    # For each create an FilteredReadIndexedBam object
    subsample_bam = FilteredReadIndexedBam(bam_path, subsample)

    # Compute the samples_metrics for this subsample
    print(f"Computing sample metrics for IVT subsample number  {i + 1}")
    samples_metrics_IVT_norm_subsample, all_bam_reads = io.get_ref_reg_samples_metrics(
        ref_reg,
        [(pod5_dr_IVT, subsample_bam)],
        metric="dwell_trimmean_trimsd",
        sig_map_refiner=sig_map_refiner,
        signal_type='norm'
        )

    # Calculate the MWU-statistic for Ctrl vs. this subsample of IVT
    print("Computing the MWU-statistics for Ctrl with this subsample of IVT-data")
    MWU_results_subsample_df, MWU_sig_positions_subsample = compute_MWU_results(
        ref_reg,
        samples_metrics_1 = samples_metrics_bio,
        samples_metrics_2 = samples_metrics_IVT_norm_subsample,
        min_reads = 30)
    if MWU_sig_positions_subsample != None:
        MWU_results_subsample_df['subsample'] = i  # tag which subsample this is from
        all_MWU_results.append(MWU_results_subsample_df)
print('Done with calculating all the MWU-tests')
combined_df = pd.concat(all_MWU_results, ignore_index=True)

# Count how many times each position is significant
position_counts = combined_df['position'].value_counts()

# Filter all results for only the stable positions, i.e. positions which returned significant in 90% of the bootstrapped subsamples
stable_positions = position_counts[position_counts >= 0.9 * 50].index  # 90% of 50 subsamples
stable_results = combined_df[combined_df['position'].isin(stable_positions)]

# Summarize, e.g., mean and std of p-value, RBC, and medians
print('Summarizing the MWU-results')
summary = stable_results.groupby('position').agg({
    'p_value': ['mean', 'std'],
    'adjusted_p_value': ['mean', 'std'],
    'RBC': ['mean', 'std'],
    'median_trimmean_1': ['mean', 'std'],
    'median_trimmean_2': ['mean', 'std'],
    'num_sample_1': 'mean',
    'num_sample_2': 'mean'
}).reset_index()
summary.columns = [
    'position',
    'pval_mean', 'pval_std',
    'adjusted_pval_mean', 'adjusted_pval_std',
    'RBC_mean', 'RBC_std',
    'median_trimmean_1_mean', 'median_trimmean_1_std',
    'median_trimmean_2_mean', 'median_trimmean_2_std',
    'num_sample_1_mean',
    'num_sample_2_mean'
]

end_time = datetime.now()
print(f"Ended at: {end_time}")

In [ ]:
print(f"MWU-testing results:")

print(f"\tThe number of significant positions: {len(list(summary['position'].values))}")
print(f"\tThe significant positions: {list(summary['position'].values)}")

#### Export these results

In [ ]:
summary.to_csv("path/to/data/Remora_results.csv", index=False)

### MWU testing without bootstrapping

In [ ]:
MWU_results_df, MWU_sig_positions = compute_MWU_results(
        ref_reg,
        samples_metrics_1 = samples_metrics_bio,
        samples_metrics_2 = samples_metrics_IVT,
        min_reads = )
print(f"\nMWU-testing results without bootstrapping:")
print(f"\tThe number of significant positions: {len(MWU_sig_positions)}")
print(f"\tThe significant positions: {list(MWU_sig_positions)}")

## 6. Visualisation

### Some constants to set for plotting

Define the colours to use in plotting

In [ ]:
color_bio = "#009E73"
color_IVT = "#7851A9"

Define the sequence of the reference regions of interest

In [ ]:
seq_Glu_CTC_1_1 = "TCCCTGGTGGTCTAGTGGTTAGGATTCGGCGCTCTCACCGCCGCGGCCCGGGTTCGATTCCCGGTCAGGGAA"
seq_Gly_GCC_2_1 = "GCATTGGTGGTTCAGTGGTAGAATTCTCGCCTGCCACGCGGGAGGCCCGGGTTCGATTCCCGGCCAATGCA"
seq_Leu_CAA_1_1_WI = "GTCAGGATGGCCGAGTGGTCTAAGGCGCCAGACTCAAGTTCTGGTCTCCAATGGAGGCGTGGGTTCGAATCCCACTTCTGACA"
seq_Leu_CAA_1_1 = "GTCAGGATGGCCGAGTGGTCTAAGGCGCCAGACTCAAGCTAAGCTTCCTCCGCGGTGGGGATTCTGGTCTCCAATGGAGGCGTGGGTTCGAATCCCACTTCTGACA"
seq_Leu_CAA_1_2 = "GTCAGGATGGCCGAGTGGTCTAAGGCGCCAGACTCAAGCTTGGCTTCCTCGTGTTGAGGATTCTGGTCTCCAATGGAGGCGTGGGTTCGAATCCCACTTCTGACA"

### Line plots: the trimmean signal levels per read

#### Biological sample

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")
plot_trimmean_signals_giving_metrics_one_sample_with_seq(
    ref_reg,
    samples_metrics_bio,
    xlim_start = 0,
    xlim_end = ,
    max_reads = None,
    signal_type='norm', 
    color = color_bio, 
    alpha_lines = 1/10, 
    highlight_positions = None, 
    sequence = )
end_time = datetime.now()
print(f"Ended at: {end_time}")

#### IVT sample

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")
plot_trimmean_signals_giving_metrics_one_sample_with_seq(
    ref_reg,
    samples_metrics_IVT, 
    xlim_start = 0, 
    xlim_end = ,
    max_reads = ,
    signal_type='norm', 
    color = color_IVT,
    alpha_lines = 1/20, 
    sequence = ,
    highlight_positions = [])
end_time = datetime.now()
print(f"Ended at: {end_time}")

#### Bio vs. IVT sample

In [ ]:
start_time = datetime.now()
print(f"Started at: {start_time}")
plot_trimmean_signals_two_samples_with_seq(
    ref_reg,
    samples_metrics_1 = samples_metrics_bio,
    samples_metrics_2 = samples_metrics_IVT,
    xlim_start=0,
    xlim_end=,
    signal_type='norm',
    max_reads=,
    alpha_lines=1/15,
    sample_labels=['Biological sample',"IVT sample"], sequence =,
    highlight_positions = None
)
end_time = datetime.now()
print(f"Ended at: {end_time}")

### Scatterplots: mean and std of trimmed mean signal levels

In [ ]:
plot_mean_and_sd_of_trimmean_two_samples(
    ref_reg,
    samples_metrics_1 = samples_metrics_bio,
    samples_metrics_2 = samples_metrics_IVT,
    xlim_start = 0,
    xlim_end = ,
    sample_labels=['Biological',"IVT"],
    signal_type="norm"
)

### Violin plots

In [ ]:
plot_trimmean_violin_two_samples_with_seq(
    ref_reg = ref_reg,
    samples_metrics_1= samples_metrics_bio,
    samples_metrics_2 = samples_metrics_IVT,
    sample_labels=['Biological',"IVT"],
    xlim_start=0,
    xlim_end=,
    signal_type='norm',
    highlight_positions_one_ionic = [],
    highlight_positions_two_ionic = [],
    highlight_positions_three = [],
    highlight_positions_basecalled = [],
    sequence = , legend = 'no', title = 'Remora: '
)

In [ ]:
plot_trimmean_violin_two_samples_with_seq(
    ref_reg = ref_reg,
    samples_metrics_1= samples_metrics_bio,
    samples_metrics_2 = samples_metrics_IVT ,
    sample_labels=["Biological","IVT"],
    xlim_start=0,
    xlim_end= ,
    signal_type='norm',

    highlight_positions_one_ionic=[],
    highlight_positions_two_ionic = [],
    highlight_positions_three = [],
    highlight_positions_basecalled = [],
    highlight_positions_excluded = [],
    sequence = seq_Glu_CTC_1_1, legend = 'no', title = 'Remora:'
)